# Lab Exercise 2: Design and implementation of FIR digital filters using Python
The purpose of the second set of exercises is to become familiar with the functions for designing finite impulse response (FIR) filters and implementing them in Python. Before you begin the exercise, you should carefully study <a href='https://helios.ntua.gr/pluginfile.php/260139/mod_resource/content/2/chapter_01-en.pdf'>Chapter 1</a> and, in particular, Section 1.3 of the course booklet. 

In [ ]:
# Import libraries needed for the exercise

# Import signal processing functions from SciPy
import scipy
from scipy import signal, io
from scipy.signal import firwin2, welch, convolve
from scipy.fft import fft, fftfreq, ifft
from scipy.fftpack import fftshift, ifftshift

# Import Matplotlib for plotting graphs
import matplotlib
import matplotlib.pyplot as plt

# Import NumPy for numerical operations
import numpy as np
from numpy import random

# Print a message indicating successful import of libraries
print("Libraries added successfully!")

## Part 1, Introduction to fft and ifft functions
Try the following in the command window to consolidate the use of the $fftshift$ and $ifftshift$ functions.

In [ ]:
# Create the array X
X = np.arange(-2, 3)  
# Apply fftshift and ifftshift
fftshifted_X = np.fft.fftshift(X)
ifftshifted_X = np.fft.ifftshift(X)
# Double fftshift and a combination of fftshift and ifftshift
Y = np.fft.fftshift(np.fft.fftshift(X))
Z = np.fft.ifftshift(np.fft.fftshift(X))
print('X=',X) 
print('Y=',Y) 
print('Z=',Z) 


<b>Question 1</b>: Which of the vectors Y and Z equals X? 

<b>Question 2</b>: Repeat with $X=[-1:2]$. What do you observe? 

Try the following two examples to consolidate the use of the $fftshift$ and $ifftshift$ functions in combination with $fft$ and $ifft$.

In [ ]:
# First part: Original signal and its FFT
xb = np.array([1, 2, 3, 4, 5, 4, 3, 2, 1])
x = ifftshift(xb)
X = fft(x)
Xb = fftshift(X)  # Spectrum with DC component in the center

# Second part: Low-pass filter effect
Xb_low_pass = np.array([0, 0, 1, 1, 1, 1, 1, 0, 0])
X_low_pass = ifftshift(Xb_low_pass)
x_low_pass = ifft(X_low_pass)
xb_low_pass = fftshift(x_low_pass.real)

<b>Question 3</b>: Modify the previous example so that you start directly with the definition of the spectrum of the low-pass signal $X$ as expected by the $ifft$. Write your answer.

## Part 2: Design and implementation of filters

In this Part of the exercise you will examine two alternative methods for designing FIR filters:
<b>a) the window method, and
b) the equal ripple method.</b>

In the following example, we have implemented a low pass filter with cut-off frequency Fs/8 applied it on a real signal, parameter s, which is stored in the <a href='https://helios.ntua.gr/mod/resource/view.php?id=5308'>file sima.mat</a> (a binary MATLAB file). This is a sonar signal with a spectrum extending up to about 4 KHz and a sampling frequency of Fs=8192 (it is also stored in the sima.mat file, along with the signal). Note that, in the code of the example, the loading of signal s precedes the rest of the code, allowing the use of the variables s, Fs.

In [ ]:
# Sonar Signal 
# import sima.mat, matlab data file
mat = io.loadmat('sima.mat')
Fs = mat.get('Fs')[0].item()
s = np.array(mat.get('s')).flatten()

f, Pxx = scipy.signal.welch(s, Fs)
Pxx = 10 * np.log10(abs(Pxx))
plt.figure(); plt.plot(f, Pxx); plt.grid(True); plt.xlabel('Frequency [Hz]'); plt.ylabel('PSD dB/Hz')

N = Fs #Adjusted for Python indexing, assuming Fs is even
H = np.concatenate((np.ones(N//8), np.zeros(3*N//4), np.ones(N//8)))

In [ ]:
# Use Alternative Signal s of 4 sunisoidal signals
Fs = 8192        # Sampling frequency
Ts = 1/Fs
T = 1  # Signal duration

# Create signal 

In [ ]:
h = np.fft.ifft(H, n=N).real
middle=len(h)//2
h=np.concatenate((h[middle+1:N+1], h[0:middle+1]), casting="same_kind") # use fftshift(h) to replace this line of code  

h32 = h[N//2-16:N//2+17]
h64 = h[N//2-32:N//2+33]
h128 = h[N//2-64:N//2+65]

plt.figure()
plt.stem(range(len(h64)), h64, basefmt=" ")
plt.title('Stem Plot of h64')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.grid()

w, h = scipy.signal.freqz(h64)
plt.figure()
plt.plot(w/np.pi , np.abs(h))  # Normalizing frequency to Fs 
plt.title('Frequency Response of h64')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.grid()

In [ ]:
plt.figure(figsize=(14, 4))

for filt, label in zip([h32, h64, h128], ['h32', 'h64', 'h128']):
    w, h = scipy.signal.freqz(filt)
    plt.plot(w / np.pi, 20*np.log10(np.abs(h)), label=label)

plt.title('Frequency Responses of h32, h64, h128')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.legend()
plt.grid()
plt.grid(True)
plt.show()

wh = np.hamming(len(h64))
wk = np.kaiser(len(h64), 5)
plt.figure(); plt.plot(wh, 'b', label='Hamming'); plt.plot(wk, 'r', label='Kaiser'); plt.legend(); plt.grid()

h_hamming = h64 * wh
h_kaiser = h64 * wk

# Plot the impulse responses
plt.figure(figsize=(10, 6))
markers, stemlines, baseline = plt.stem(range(len(h64)), h64, linefmt='C0-', markerfmt='C0o', basefmt="C0", label='h64')
plt.setp(markers, markersize = 5)  # Adjusting marker size for better visibility
plt.setp(stemlines, linestyle = '-')  # Adjusting stem line style

markers, stemlines, baseline = plt.stem(range(len(h_hamming)), h_hamming, linefmt='C1-', markerfmt='C1o', basefmt="C1", label='h_hamming')
plt.setp(markers, markersize = 5)
plt.setp(stemlines, linestyle = '-')

markers, stemlines, baseline = plt.stem(range(len(h_kaiser)), h_kaiser, linefmt='C2-', markerfmt='C2o', basefmt="C2", label='h_kaiser')
plt.setp(markers, markersize = 5)
plt.setp(stemlines, linestyle = '-')

plt.title('Impulse Responses')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot the frequency responses
plt.figure(figsize=(14, 4))
for filt, label, color in zip([h64, h_hamming, h_kaiser], ['h64', 'h_hamming', 'h_kaiser'], ['C0', 'C1', 'C2']):
    w, h = scipy.signal.freqz(filt)
    plt.plot(w / np.pi, 20 * np.log10(np.abs(h)), label=label, color=color)

plt.title('Frequency Responses')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Filtering and plotting PSD
y_rect = np.convolve(s, h64, mode='same')
f, Pxx = scipy.signal.welch(y_rect, Fs)
Pxx = 10 * np.log10(abs(Pxx))
plt.figure();plt.grid(True); plt.plot(f, Pxx); plt.xlabel('Frequency [Hz]'); plt.ylabel('PSD dB/Hz'); plt.title('h64')

y_hamm = np.convolve(s, h_hamming, mode='same')
f, Pxx = scipy.signal.welch(y_hamm, Fs)
Pxx = 10 * np.log10(abs(Pxx))
plt.figure();plt.grid(True); plt.plot(f, Pxx); plt.xlabel('Frequency [Hz]'); plt.ylabel('PSD dB/Hz'); plt.title('h_hamming')

y_kais = np.convolve(s, h_kaiser, mode='same')
f, Pxx = scipy.signal.welch(y_kais, Fs)
Pxx = 10 * np.log10(abs(Pxx))
plt.figure(); plt.grid(True); plt.plot(f, Pxx); plt.xlabel('Frequency [Hz]'); plt.ylabel('PSD dB/Hz'); plt.title('h_kaiser')

In [ ]:
# Parks-McClellan low-pass filter
freq = np.array([0, 0.10, 0.15, 0.5])*(Fs)  # Scale frequencies correctly
gain = [1, 1, 0, 0]

# Call firwin2 with corrected parameters and using the `fs` parameter
#hpm = signal.firwin2(128, freq, gain, fs=Fs)
hpm = firwin2(128, freq, gain, fs=Fs)
s_pm = np.convolve(s, hpm, 'same')

w, h = scipy.signal.freqz(hpm)
plt.figure()
plt.plot(w / np.pi, np.abs(h))  # Normalizing frequency to Hz
plt.title('Frequency Response of Parks McClellan')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.grid()

f, Pxx = scipy.signal.welch(s_pm, Fs)
Pxx = 10 * np.log10(abs(Pxx))
plt.figure();plt.grid(True); plt.plot(f, Pxx); plt.xlabel('Frequency [Hz]'); plt.ylabel('PSD dB/Hz'); plt.title('Signal with Parks McClellan filtering')

<b>Modify the code to experiment:</b>
1. Replace h=np.concatenate((h[middle+1:N+1], h[0:middle+1]), casting="same_kind") so that you achieve the same result by using either the ifftshift or fftshift function.

2. Modify the code so that a low-pass <b>filter of length 140+1 </b> is used instead of 64+1. Plot the filter’s frequency response using the window-design method, as in the example, for two cases: rectangular and Hamming windows.

3. Design a <b> Parks–McClellan filter of length 140+1 </b>, with the same edge frequencies as in the example (0.1, 0.15). Plot the frequency response of this filter as well and comment on the differences from the same filter of length 64+1. Write your answers.

4. Change the edge frequencies of the filter from question 3 to <b>(0.11, 0.12) </b> and compare the frequency responses of the two filters. Save the plots and write your comments.

5. Replace the signal s with the sum of four sunisoidal signals of <b> frequencies 500, 1000, 1600 and 3000 Hz </b>  and duration 1.0 s. Keep the same sampling rate Fs = 8192 Hz. Plot the signal’s power spectral density (using pwelch) and observe the effect of filtering with the two Parks–McClellan filters from questions 3 and 4. 

## Part 3: Application A
1. The purpose of Application A is to design a band-pass filter with <b> passband (0.7 KHz, 1.2 KHz)</b>  using the two methods described above, and to apply both filters to the signal s. Verify that your filter is designed correctly by checking both its frequency response and the result of filtering the signal s.
2. For the band-pass filter from question 1 above, designed using the Parks–McClellan method, select an appropriate filter order such that <b> attenuation of at least 50 dB </b>is achieved in the stopbands. Apply this filter to the four-tone sinusoidal signal described in question 5 of Part 2.

In [ ]:
# Create a band-pass filter in the frequency domain
N = Fs  # Number of frequency bins
f1=700
f2=1200

In [ ]:
# Design a Parks-McClellan band-pass filter with 50db Attenuation


## Part 4: Application B
Design and implement a filter, as in Application A, with two passbands: (0 Hz, 500 Hz) and (1100 Hz, 3000 Hz).

In [ ]:
# Create a dual-zone band-pass filter in the frequency domain
